In [7]:
import pandas as pd
import numpy as np

df = pd.read_csv('weatherAUS.csv')

In [25]:
df["Temp3pm"].describe()

count    141851.00000
mean         21.68339
std           6.93665
min          -5.40000
25%          16.60000
50%          21.10000
75%          26.40000
max          46.70000
Name: Temp3pm, dtype: float64

In [9]:
df.corr(numeric_only=True)["RainTomorrow"]

MinTemp          0.083936
MaxTemp         -0.159237
Rainfall         0.239032
Evaporation     -0.119285
Sunshine        -0.450768
WindGustSpeed    0.234010
WindSpeed9am     0.090995
WindSpeed3pm     0.087817
Humidity9am      0.257161
Humidity3pm      0.446160
Pressure9am     -0.246371
Pressure3pm     -0.226031
Cloud9am         0.317380
Cloud3pm         0.381870
Temp9am         -0.025691
Temp3pm         -0.192424
RainTomorrow     1.000000
Name: RainTomorrow, dtype: float64

## Usa arvore de decisão com entropia para decidir os cortes para discretização de variáveis

In [38]:
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier

# Exemplo de dados
# X: Variável numérica (ex: idade)
# y: Variável alvo da Rede Bayesiana (ex: Doença Sim/Não)
df = pd.read_csv("weatherAUS.csv")

df["RainTomorrow"] = df["RainTomorrow"].map({"No": 0, "Yes": 1})

total = len(df)

vars_to_use = ["Humidity3pm", "Pressure3pm", "WindGustSpeed"]

df.dropna(subset=[*vars_to_use, "RainTomorrow"], inplace=True)

print(f"Total de amostras após remoção de NaNs: {len(df)} (antes: {total})")

for var in vars_to_use:
    X = df[[var]]  # Precisa ser 2D
    y = df["RainTomorrow"]

    # 1. Configurar a Árvore
    # max_leaf_nodes determina o número máximo de categorias (bins) que você quer.
    # Para Redes Bayesianas, mantenha baixo (ex: 3 a 5) para não explodir a Tabela de Probabilidade Condicional (CPT).
    arvore = DecisionTreeClassifier(
        criterion="entropy",  # Crucial: usar entropia para maximizar ganho de informação
        max_leaf_nodes=4,  # Isso gerará até 4 categorias (3 cortes)
    )

    # 2. Treinar
    arvore.fit(X, y)

    # 3. Extrair os pontos de corte (Thresholds)
    # A árvore armazena os cortes na estrutura interna. Vamos extraí-los:
    thresholds = arvore.tree_.threshold
    thresholds = thresholds[thresholds != -2]  # -2 indica nós folhas (sem corte)
    thresholds = np.sort(thresholds)

    print(f"Pontos de corte ótimos baseados em entropia para {var}: {thresholds}")

    # 4. Aplicar os cortes para criar a nova variável categórica
    df[var + "_cat"] = pd.cut(
        df[var], bins=[-np.inf] + list(thresholds) + [np.inf], labels=False
    )
cat_vars_labels = [var + "_cat" for var in vars_to_use]
df_cat = df[cat_vars_labels + ["RainTomorrow"]]
print(df_cat.head())

Total de amostras após remoção de NaNs: 121166 (antes: 145460)
Pontos de corte ótimos baseados em entropia para Humidity3pm: [52.5 66.5 81.5]
Pontos de corte ótimos baseados em entropia para Pressure3pm: [1005.25       1011.04998779 1018.75      ]
Pontos de corte ótimos baseados em entropia para WindGustSpeed: [36. 47. 58.]
   Humidity3pm_cat  Pressure3pm_cat  WindGustSpeed_cat  RainTomorrow
0                0                1                  1           0.0
1                0                1                  1           0.0
2                0                1                  1           0.0
3                0                2                  0           0.0
4                0                1                  1           0.0


In [ ]:
import pandas as pd
from pgmpy.models import BayesianNetwork
from pgmpy.estimators import MaximumLikelihoodEstimator

# 1. Definir a Estrutura (Topologia)
# Você lista as arestas: (Causa -> Efeito)
model = BayesianNetwork([
    ('Pressure3pm_cat', 'RainTomorrow'),
    ('Humidity3pm_cat', 'RainTomorrow'),
    ('WindGustSpeed_cat', 'RainTomorrow'),
    # Se quiser adicionar complexidade (ex: Pressão afeta Vento):
    # ('Pressure3pm_cat', 'WindGustSpeed_cat') 
])

# 2. Verificar se o modelo é válido (sem ciclos)
# Ex: A->B->A não é permitido
assert model.check_model()

# 3. "Aprender" as CPTs (Tabelas de Probabilidade Condicional)
# Aqui o pgmpy vai contar as frequências no seu DF discretizado
# e calcular P(Chuva | Pressão, Umidade, Vento)
model.fit(df, estimator=MaximumLikelihoodEstimator)

# 4. Visualizar uma das tabelas (Ex: Probabilidade de Chuva)
print("CPT da variável RainTomorrow:")
print(model.get_cpds('RainTomorrow'))

In [39]:
df_cat.corr()["RainTomorrow"]

Humidity3pm_cat      0.482644
Pressure3pm_cat     -0.222487
WindGustSpeed_cat    0.231240
RainTomorrow         1.000000
Name: RainTomorrow, dtype: float64